<a href="https://colab.research.google.com/github/ThisuriCham/Pytorch/blob/master/training_classifier_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torchvision as tv
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader


In [0]:
!pip3 install http://download.pytorch.org/whl/cu80/torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl

In [0]:
!pip3 install torchvision

In [0]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

In [12]:
trainset = tv.datasets.CIFAR10(root = './data',train=True,transform=True,download=True)
trainLoader = DataLoader(trainset,batch_size=4,shuffle=True,num_workers=0)

testset = tv.datasets.CIFAR10(root = './data',train=False,transform=True,download=True)
testloader = DataLoader(testset,batch_size=4,shuffle =False,num_workers=0)

classes = ('plane', 'car', 'bird', 'cat','deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [0]:
import matplotlib.pyplot as plt
import numpy as np


In [17]:
def imshow(img):
  img = img/2+0.5
  npimg = img.numpy() 
  plt.imshow(np.transpose(npimg,(1,2,0)))
  plt.show()

dataiter = iter(trainLoader)
images, labels = dataiter.next()

imshow(tv.utils.make_grid(images))
print(''.join('%5s'%classes[labels[j]] for j in range(4)))


TypeError: ignored

TypeError: ignored